In [1]:
# from re import S
import tensorflow as tf

from VideoSwinTransformer import SwinTransformer3D


swin = SwinTransformer3D()





In [2]:
x = tf.random.normal([2, 8, 32,32, 3])

output = swin(x)
print(output.shape)


(2, 768, 2, 1, 1)


In [3]:
swin.summary()

Model: "swin_transformer3d"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
patch_embed (PatchEmbed3D)   multiple                  18528     
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
basic_layer (BasicLayer)     multiple                  320426    
_________________________________________________________________
basic_layer_1 (BasicLayer)   multiple                  1211468   
_________________________________________________________________
basic_layer_2 (BasicLayer)   multiple                  11923632  
_________________________________________________________________
basic_layer_3 (BasicLayer)   multiple                  14219288  
_________________________________________________________________
layer_normalization_27 (Laye multiple           

In [7]:
input = tf.keras.Input(( 8, 32,32, 3), dtype='float64')
swin = SwinTransformer3D()
# x = 
# y = tf.keras.Dense(2, activation= "softmax")(x)
# model = tf.keras.Model(input, y)

In [ ]:
model = tf.keras.Sequential()
model.add(SwinTransformer3D())
model.add(tf.keras.layers.Dense(6))
model.compile(optimizer='sgd', loss='mse')
# This builds the model for the first time:
model.fit(x, y, batch_size=10, epochs=10)

In [13]:
x = tf.random.normal([100,8, 32,32, 3])
y = tf.random.uniform(shape=[100], minval=0, maxval=5, dtype='int64')
y

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([1.5589648 , 4.131706  , 3.424728  , 0.03354549, 3.9374757 ,
       1.9532555 , 1.4631528 , 4.9608464 , 4.790521  , 2.7811658 ,
       0.8233148 , 0.67228556, 0.66149294, 2.6740491 , 2.8545089 ,
       2.5485253 , 2.4126172 , 0.77902675, 1.8516135 , 2.4605334 ,
       2.8350801 , 1.038543  , 0.9111941 , 4.994187  , 1.8475294 ,
       1.8963927 , 3.8615584 , 3.4105873 , 1.9966418 , 3.9203568 ,
       3.3940077 , 3.6697948 , 2.760222  , 0.54742575, 3.2439911 ,
       4.9453897 , 4.101827  , 3.5235178 , 4.7893124 , 0.11487126,
       4.6799335 , 3.256632  , 1.5831792 , 0.0055778 , 4.6060953 ,
       1.911403  , 3.8623018 , 4.57571   , 2.8755665 , 3.96671   ,
       2.1448815 , 0.95594823, 3.726253  , 2.0881004 , 4.0867558 ,
       1.0058558 , 3.2285786 , 0.82421184, 2.2420616 , 3.028944  ,
       4.080575  , 2.0648265 , 1.316492  , 0.95437944, 3.8159304 ,
       4.1701217 , 2.4723864 , 3.4331596 , 1.501295  , 1.3364708 ,
       4.16945

In [ ]:
x  = swin(x)
print(x.shape)

In [14]:
# swin.summary()

In [3]:
tf.constant(-100, dtype= 'float64')

<tf.Tensor: shape=(), dtype=float64, numpy=-100.0>